In [7]:
#--requrement
#python==10
#pip install selenium-wire
#!python.exe -m pip install --upgrade pip
#!pip install blinker==1.7.0

  Using cached blinker-1.7.0-py3-none-any.whl.metadata (1.9 kB)
Using cached blinker-1.7.0-py3-none-any.whl (13 kB)
  Attempting uninstall: blinker
    Found existing installation: blinker 1.9.0
    Uninstalling blinker-1.9.0:
      Successfully uninstalled blinker-1.9.0


In [1]:
from seleniumwire import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import time

import requests
import re
import csv

In [2]:
# Function to click download button and capture CSV URLs
def capture_csv_urls(button_id):
    print(f"\n Looking for button with ID: {button_id}")
    # Clear previous requests
    del driver.requests  
    # Find and click the download button
    download_button = driver.find_element("id", button_id)
    download_button.click()
        
    time.sleep(5)  # Wait for requests to fire
        
    # Extract new CSV URLs
    new_urls = []
    for request in driver.requests:
        if (
                request.response
                and request.method == 'GET'
                and '/download/csv/' in request.url
                and request.response.status_code == 200 # code change as per the site
            ):
            print(f"Found CSV URL: {request.url}")
            new_urls.append(request.url)
    return new_urls


In [18]:
df=pd.read_csv(r"C:\Users\I33238\Downloads\Remaining_links.csv") 
#this the csv which contain csv with http links generated from the script
d1=[]
for index, row in df.iterrows():
    
    options = {'request_storage': 'memory', } # Store requests in memory
    driver = webdriver.Chrome(seleniumwire_options=options)

    # Visit the page
    driver.get(row['links'])
    time.sleep(5)
    csv_urls = []
    try:
        csv_urls += capture_csv_urls("widget-download-button-naturalForest")
    except:
        csv_urls.append("NA")
        
    try:
        csv_urls += capture_csv_urls("widget-download-button-treeCoverPlantations")   
    except:
        csv_urls.append("NA")
    
    try:
        driver.get(row['links'].replace("land-cover","fires"))
        time.sleep(5)
        csv_urls += capture_csv_urls("widget-download-button-treeLossFiresAnnual")   
    except:
        csv_urls.append("NA")
        
    try:    
        d1.append([row["Regions"],row["provinces"],row['links'],csv_urls[0],csv_urls[2],csv_urls[3]])
    except:
        d1.append([row["Regions"],row["provinces"],row['links'],csv_urls[0],csv_urls[1],csv_urls[2]])
    
driver.quit()


🔍 Looking for button with ID: widget-download-button-naturalForest
Found CSV URL: https://www.globalforestwatch.org/api/data/dataset/gadm__tcl__adm2_summary/v20250515/download/csv/?sql=SELECT%20iso%2C%20adm1%2C%20adm2%2C%20sbtn_natural_forests__class%2C%20SUM(area__ha)%20AS%20area__ha%20FROM%20data%20WHERE%20iso%20%3D%20%27CHN%27%20AND%20adm1%20%3D%20998%20AND%20adm2%20%3D%201%20AND%20umd_tree_cover_density_2000__threshold%20%3D%200%20%20GROUP%20BY%20iso%2C%20adm1%2C%20adm2%2C%20sbtn_natural_forests__class

🔍 Looking for button with ID: widget-download-button-treeCoverPlantations

🔍 Looking for button with ID: widget-download-button-treeLossFiresAnnual
Found CSV URL: https://www.globalforestwatch.org/api/data/dataset/gadm__tcl__adm2_change/v20250515/download/csv/?sql=SELECT%20iso%2C%20adm1%2C%20adm2%2C%20umd_tree_cover_loss__year%2C%20SUM(umd_tree_cover_loss__ha)%20AS%20umd_tree_cover_loss__ha%2C%20SUM(umd_tree_cover_loss_from_fires__ha)%20AS%20%22umd_tree_cover_loss_from_fires__ha%22

In [19]:
#just provide the name and location to save csv
with open(r"C:\Users\I33238\Downloads\province_links_nf3.csv", mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Regions","provinces", "links","naturalForest","treeCoverPlantations","treeLossFiresAnnual"])  # Header ,"treeCoverPlantations" "naturalForest"
    writer.writerows(d1)

print("CSV file 'province_links.csv' saved successfully.")

CSV file 'province_links.csv' saved successfully.
